In [1]:
import pandas as pd
import sys
sys.path.insert(0,r"C:\Users\LENOVO\Downloads")
import numpy as np
import csv
from sklearn.model_selection import train_test_split
import class_report as cr
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score, auc, f1_score, log_loss
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

In [2]:
pd.set_option('display.max_row', None)
pd.set_option('display.max_column', None)

In [3]:
base_eje = pd.read_csv(r'C:\Users\LENOVO\Desktop\base_eje.txt', sep = ",", engine = "python")

In [4]:
base2_eje = base_eje[['SUSCNITC','SERVDESC','SESUCUSA','SESUCICL','SESULOCA',
                                  'TIPIFICACION_CLIENTE','CALIFICACION_VECTOR_SERVICIO',
                                  'CANAL_ENVIO','SEGMENTACION','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR',
                                  'ANTIGUEDAD_DIAS','CANAL_PAGO','SESUSAPE','SESUSAAN','PROM_SUSC']]
base3_eje = base2_eje.dropna()

In [5]:
tmp = base3_eje[['SUSCNITC','CALIFICACION_VECTOR_SERVICIO']]
tmp.columns

group_nit = tmp.groupby(['SUSCNITC'])['CALIFICACION_VECTOR_SERVICIO'].mean().reset_index()
group_nit = group_nit.rename(columns={"CALIFICACION_VECTOR_SERVICIO":"PROM_CALIFICACION"})

In [6]:
base4_eje = pd.merge(base3_eje, group_nit, how='left', on=['SUSCNITC'])

In [7]:
base4_eje['y'] = pd.cut(x=base4_eje['PROM_CALIFICACION'],
                            bins=[-1,50,76,100],
                            labels=['No Pago','Pago Inoportuno','Pago'])

In [8]:
base5_eje = base4_eje[['SERVDESC','SESUCUSA','SESUCICL','SESULOCA','TIPIFICACION_CLIENTE',
                       'CALIFICACION_VECTOR_SERVICIO','CANAL_ENVIO','SEGMENTACION','ESTRATO_AGRUPADO',
                       'DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS','CANAL_PAGO','SESUSAPE','SESUSAAN','PROM_SUSC','y']]

In [9]:
names = base5_eje.columns

for x in names:
    globals()['FREQ_%s' % x]= base5_eje[x].value_counts().rename_axis([x]).reset_index(name='F_{x}'.format(x=x))

In [10]:
freq_eje = base5_eje.merge(FREQ_SERVDESC,on="SERVDESC").merge(FREQ_SESUCUSA,on='SESUCUSA').merge(FREQ_SESUCICL,on='SESUCICL').merge(FREQ_SESULOCA,on='SESULOCA').merge(FREQ_TIPIFICACION_CLIENTE,on='TIPIFICACION_CLIENTE').merge(FREQ_CANAL_ENVIO,on='CANAL_ENVIO').merge(FREQ_SEGMENTACION,on='SEGMENTACION').merge(FREQ_ESTRATO_AGRUPADO,on='ESTRATO_AGRUPADO').merge(FREQ_DEPARTAMENTO_AGR,on='DEPARTAMENTO_AGR').merge(FREQ_CANAL_PAGO,on='CANAL_PAGO')

In [11]:
freq_eje2 = freq_eje[['F_SERVDESC','F_SESUCUSA','F_SESUCICL','F_SESULOCA','F_TIPIFICACION_CLIENTE',
                      'F_CANAL_ENVIO','F_SEGMENTACION','F_ESTRATO_AGRUPADO','F_DEPARTAMENTO_AGR',
                      'F_CANAL_PAGO']]

In [12]:
freq_eje2['AVG_FREQ'] = freq_eje2.mean(axis=1)

C:\Users\LENOVO\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
n, m = freq_eje2.shape
n_reg_del = round((0.05 * n) /100)
outliers = freq_eje2.nsmallest(n_reg_del, 'AVG_FREQ')
outliers_idx = outliers.index
outliers_idx = outliers_idx.sort_values(ascending=False)

In [14]:
for x in outliers_idx:
    base5_eje = base5_eje.drop([x])

In [24]:
dataTypeSeries = base5_eje.dtypes
dataTypeSeries

cat = base5_eje.select_dtypes(include=['object','category'])
cuanti = base5_eje.select_dtypes(include=['int64','float64'])
  
cat = cat.apply(lambda col: LabelEncoder().fit_transform(col))
base5_eje = cuanti.join(cat)

In [25]:
X = base5_eje[['SERVDESC','SESUCUSA','SESUCICL','SESULOCA','TIPIFICACION_CLIENTE','CANAL_ENVIO',
               'SEGMENTACION','ESTRATO_AGRUPADO','DEPARTAMENTO_AGR','ANTIGUEDAD_DIAS','CANAL_PAGO',
               'SESUSAPE','SESUSAAN','PROM_SUSC']]

y = base5_eje['y']

In [26]:
from datetime import datetime
instanteInicial = datetime.now()
X_ = StandardScaler().fit_transform(X)
#X_ = X
X_train, X_test, y_train, y_test = train_test_split(X_, y, test_size=0.2,random_state=321)

In [28]:
kf = StratifiedKFold(n_splits=5,shuffle=False)

accuracy_model = []
# Iteraciones del entrenamiento sobre los kfolds
#X_fin = X[['SESUSERV','SESUCUSA', 'SESUSAPE','SESUCICL', 'TIPIFICACION_CLIENTE', 'PROM_SUSC',
#'CANAL_ENVIO','SEGMENTACION', 'ESTRATO_AGRUPADO','ANTIGUEDAD_DIAS', 'CANAL_PAGO']]
X_fin = X
X_ = StandardScaler().fit_transform(X_fin)

for train_index, test_index in kf.split(X_, y) :
    # Particion train test para validacion cruzada
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    # Entrenamiento de la red MLP
    model = KNeighborsClassifier(7).fit(X_train, y_train) 
    accuracy_model.append(cr.class_report(y_true = y_test, 
                                       y_pred = model.predict(X_test),
                                       y_score = model.predict_proba(X_test)))
 
# Print the accuracy    
print(accuracy_model)

[             precision    recall  f1-score   support      pred       AUC
0             0.918804  0.976557  0.946801   96234.0  102283.0  0.823639
1             0.645606  0.643586  0.644594    7351.0    7328.0  0.894136
2             0.409866  0.128658  0.195841    8783.0    2757.0  0.653856
avg / total   0.861151  0.888500  0.868333  112368.0  112368.0  0.948530,              precision    recall  f1-score   support      pred       AUC
0             0.914280  0.971673  0.942103   96234.0  102275.0  0.808849
1             0.629157  0.597062  0.612689    7351.0    6976.0  0.886682
2             0.381777  0.135489  0.200000    8783.0    3117.0  0.646828
avg / total   0.854006  0.881808  0.862549  112368.0  112368.0  0.944982,              precision    recall  f1-score   support      pred       AUC
0             0.927888  0.974999  0.950860   96235.0  101121.0  0.847470
1             0.526584  0.604952  0.563054    7351.0    8445.0  0.877736
2             0.356888  0.113869  0.172652    87

In [29]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, model.predict(X_test))

array([[93640,  1259,  1336],
       [ 2248,  4530,   572],
       [ 5400,  2142,  1241]], dtype=int64)